In [1]:
import re
import nltk
import pandas as pd
import numpy as np

# Taller 2
## Token extraction

In [57]:
files = [
    '../Files/allan_poe/decenso_al_maelstron.txt',
    '../Files/allan_poe/lady_ligeia.txt',
    '../Files/allan_poe/berenice.txt',
    '../Files/allan_poe/la_esfinje.txt',
    '../Files/allan_poe/morerlla.txt',
    '../Files/allan_poe/el_retrato_oval.txt',
    '../Files/allan_poe/william_wilson.txt',
    '../Files/allan_poe/el_engaño_del_globo.txt',
    '../Files/allan_poe/la_máscara_de_la_muerte_roja.txt',

    # '../Files/cien_soledad.txt',
    # '../Files/el_principito.txt',
    # '../Files/la_isla_del_tesoro.txt',
]
texts = []
for file_name in files:
    full_file_name = ('{}'
        .format(file_name))

    print(full_file_name)

    file = open(full_file_name, 'r')
    texts += [file.read()]
    file.close()

texts = pd.Series(texts)
text = ''.join(texts)
texts

../Files/allan_poe/decenso_al_maelstron.txt
../Files/allan_poe/lady_ligeia.txt
../Files/allan_poe/berenice.txt
../Files/allan_poe/la_esfinje.txt
../Files/allan_poe/morerlla.txt


0    Habíamos alcanzado la cumbre del despeñadero m...
1    Juro por mi alma que no puedo recordar cómo, c...
2    La desdicha es diversa. La desgracia cunde mul...
3    Durante el pavoroso reinado del cólera en Nuev...
4    Consideraba yo a mi amiga Morella con un senti...
dtype: object

In [58]:
text = text.lower()
text = re.sub(r'[-)(\s«»"]+', ' ', text) # Ignored characters
text = re.sub(r'\.\s*', '<s>', text) # Replace end of sentences

tokens = re.finditer(r'[\wáéíóúñ]+|<\/?s>|[,¿?!¡.;:]', text)
tokens = pd.Series([m.group(0) for m in tokens])

tokens

0         habíamos
1        alcanzado
2               la
3           cumbre
4              del
           ...    
21950        donde
21951      enterré
21952           la
21953      segunda
21954          <s>
Length: 21955, dtype: object

# 
## Unigrams

In [59]:
unigrams, counts = np.unique(tokens, return_counts=True)
unigrams = pd.DataFrame(counts, index=unigrams, columns=['count'])

unigrams['prob'] = unigrams['count'] / unigrams['count'].sum()
unigrams['cumsum'] = unigrams['prob'].cumsum()

unigrams

,count,prob,cumsum
!,63,0.002870,0.002870
",",1594,0.072603,0.075473
10,1,0.000046,0.075518
1645,1,0.000046,0.075564
18,1,0.000046,0.075609
...,...,...,...
último,7,0.000319,0.999453
últimos,3,0.000137,0.999590
única,2,0.000091,0.999681
único,5,0.000228,0.999909


## Bigrams

In [54]:
bigrams = np.fromiter(nltk.ngrams(tokens, 2), dtype=('<U18, <U18'))
bigrams, counts = np.unique(bigrams, return_counts=True, axis=0)

bigrams = pd.MultiIndex.from_tuples(list(bigrams), names=['w1', 'w2'])

bigrams = pd.DataFrame(counts, index= bigrams, columns=['count'])
bigrams['prfix-sum'] = bigrams['count'].groupby(level='w1').transform(np.sum)

bigrams['prob'] = bigrams['count'] / bigrams['prfix-sum']
bigrams['cumsum'] = bigrams.prob.groupby(level=['w1']).cumsum()

bigrams

count  prfix-sum      prob    cumsum
w1     w2                                              
!      ,               6        120  0.050000  0.050000
       ?               1        120  0.008333  0.058333
       a               1        120  0.008333  0.066667
       aclaró          1        120  0.008333  0.075000
       al              1        120  0.008333  0.083333
...                  ...        ...       ...       ...
útil   sobre           1         11  0.090909  1.000000
útiles <s>             1          5  0.200000  0.200000
       como            1          5  0.200000  0.400000
       domésticos      2          5  0.400000  0.800000
       y               1          5  0.200000  1.000000

[123676 rows x 4 columns]

## Trigrams

In [55]:
trigrams = np.fromiter(nltk.ngrams(tokens, 3), dtype=('<U18, <U18, <U18'))
trigrams, counts = np.unique(trigrams, return_counts=True, axis=0)

trigrams = pd.DataFrame(trigrams)
trigrams['w1'] = trigrams[['f0', 'f1']].apply(" ".join, axis=1)
trigrams['w2'] = trigrams[['f2']]
trigrams = pd.MultiIndex.from_frame(trigrams[['w1', 'w2']])

trigrams = pd.DataFrame(counts, index= trigrams, columns=['count'])
trigrams['prfix-sum'] = trigrams['count'].groupby(level='w1').transform(np.sum)

trigrams['prob'] = trigrams['count'] / trigrams['prfix-sum']
trigrams['cumsum'] = trigrams.prob.groupby(level=['w1']).cumsum()

trigrams

count  prfix-sum      prob    cumsum
w1                w2                                           
! ,               exclamó      1          6  0.166667  0.166667
                  fueron       1          6  0.166667  0.333333
                  gritó        1          6  0.166667  0.500000
                  le           1          6  0.166667  0.666667
                  no           1          6  0.166667  0.833333
...                          ...        ...       ...       ...
útiles <s>        fue          1          1  1.000000  1.000000
útiles como       hubiera      1          1  1.000000  1.000000
útiles domésticos habían       1          2  0.500000  0.500000
                  y            1          2  0.500000  1.000000
útiles y          por          1          1  1.000000  1.000000

[220696 rows x 4 columns]

## 4-grams

In [56]:
fourgrams = np.fromiter(nltk.ngrams(tokens, 4), dtype=('<U18, <U18, <U18, <U18'))
fourgrams, counts = np.unique(fourgrams, return_counts=True, axis=0)

fourgrams = pd.DataFrame(fourgrams)
fourgrams['w1'] = fourgrams[['f0', 'f1', 'f2']].apply(" ".join, axis=1)
fourgrams['w2'] = fourgrams[['f3']]
fourgrams = pd.MultiIndex.from_frame(fourgrams[['w1', 'w2']])

fourgrams = pd.DataFrame(counts, index= fourgrams, columns=['count'])
fourgrams['prfix-sum'] = fourgrams['count'].groupby(level='w1').transform(np.sum)

fourgrams['prob'] = fourgrams['count'] / fourgrams['prfix-sum']
fourgrams['cumsum'] = fourgrams.prob.groupby(level=['w1']).cumsum()

fourgrams

,,count,prfix-sum,prob,cumsum
w1,w2,,,,
"! , exclamó",<s>,1,1,1.0,1.0
"! , fueron",los,1,1,1.0,1.0
"! , gritó",una,1,1,1.0,1.0
"! , le",gritó,1,1,1.0,1.0
"! , no",había,1,1,1.0,1.0
...,...,...,...,...,...
útiles <s> fue,entonces,1,1,1.0,1.0
útiles como hubiera,podido,1,1,1.0,1.0
útiles domésticos habían,sido,1,1,1.0,1.0


In [18]:
unigrams[unigrams['cumsum'] > 0.97].iloc[0]

count     4147.000000
prob         0.027223
cumsum       0.988630
Name: y, dtype: float64